In [24]:
import cv2
import numpy as np
from glob import glob

In [44]:
def mean_image(folder_path, max_files=-1):
    image_paths = glob(folder_path + "*.tif")[:max_files]
    first_time = True
    for path in image_paths:
        print(path)
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if first_time:
            first_time = False
            mean_img = img.astype(np.float64)
        else:
            mean_img += img
    return (mean_img/len(image_paths)).astype(np.uint8)

def detect_colored_noise(image, bright_thresh_rgb=(15, 15, 15), dark_thresh=2, min_area=4):
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    diff = cv2.absdiff(image, blurred)
    b, g, r = cv2.split(diff)
    hot_b = np.uint8(b > bright_thresh_rgb[0]) * 255
    hot_g = np.uint8(g > bright_thresh_rgb[1]) * 255
    hot_r = np.uint8(r > bright_thresh_rgb[2]) * 255
    hot_rgb = cv2.bitwise_or(hot_r, cv2.bitwise_or(hot_g, hot_b))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_gray = cv2.GaussianBlur(gray, (5, 5), 0)
    diff_dark = blurred_gray.astype(np.int16) - gray.astype(np.int16)
    mask_dark = np.uint8(diff_dark > dark_thresh) * 255
    mask_dark = _filter_by_area(mask_dark, min_area)
    mask_dark = cv2.morphologyEx(mask_dark, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8))
    print("hot pixels, r: {}".format(hot_r[hot_r>0].size))
    print("hot pixels, g: {}".format(hot_g[hot_g>0].size))
    print("hot pixels, b: {}".format(hot_b[hot_b>0].size))
    print("hot pixels, rgb: {}".format(hot_rgb[hot_rgb>0].size))
    print("dark pixels: {}".format(mask_dark[mask_dark>0].size))
    return hot_rgb, (hot_b, hot_g, hot_r), mask_dark
    return mask_bright_rgb, mask_dark

def _filter_by_area(mask, min_area):
    """Filtra le maschere per area minima."""
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    filtered_mask = np.zeros_like(mask)
    for cnt in contours:
        if cv2.contourArea(cnt) > min_area:
            cv2.drawContours(filtered_mask, [cnt], -1, 255, -1)
    return filtered_mask

In [45]:
def correct_defective_pixels(image, defect_mask, kernel_size=3, method='mean'):
    if len(image.shape) == 3:
        corrected = image.copy()
        for c in range(3):
            channel = image[:, :, c]
            defect_channel = defect_mask > 0
            corrected[:, :, c] = _correct_channel(channel, defect_channel, kernel_size, method)
    else:
        corrected = _correct_channel(image, defect_mask > 0, kernel_size, method)
    return corrected

def _correct_channel(channel, defect_mask, kernel_size, method):
    corrected = channel.copy()
    defect_coords = np.argwhere(defect_mask)
    for y, x in defect_coords:
        neighborhood = channel[
            max(0, y - kernel_size//2):min(channel.shape[0], y + kernel_size//2 + 1),
            max(0, x - kernel_size//2):min(channel.shape[1], x + kernel_size//2 + 1)
        ]        
        valid_pixels = neighborhood[neighborhood != 0]  # Opzionale: escludi altri pixel difettosi se necessario
        if len(valid_pixels) > 0:
            if method == 'mean':
                corrected[y, x] = np.mean(valid_pixels)
            elif method == 'median':
                corrected[y, x] = np.median(valid_pixels)
    return corrected

In [46]:
input_path = "E:/Focus stacking/2025-04-19 - Hemiptera/A/src/"
mean_img = mean_image(input_path, 10)
hot_rgb, (hot_b, hot_g, hot_r), mask_dark = detect_colored_noise(mean_img)

E:/Focus stacking/2025-04-19 - Hemiptera/A/src\_MG_8501.tif
E:/Focus stacking/2025-04-19 - Hemiptera/A/src\_MG_8502.tif
E:/Focus stacking/2025-04-19 - Hemiptera/A/src\_MG_8503.tif
E:/Focus stacking/2025-04-19 - Hemiptera/A/src\_MG_8504.tif
E:/Focus stacking/2025-04-19 - Hemiptera/A/src\_MG_8505.tif
E:/Focus stacking/2025-04-19 - Hemiptera/A/src\_MG_8506.tif
E:/Focus stacking/2025-04-19 - Hemiptera/A/src\_MG_8507.tif
E:/Focus stacking/2025-04-19 - Hemiptera/A/src\_MG_8508.tif
E:/Focus stacking/2025-04-19 - Hemiptera/A/src\_MG_8509.tif
E:/Focus stacking/2025-04-19 - Hemiptera/A/src\_MG_8510.tif
hot pixels, r: 8
hot pixels, g: 6
hot pixels, b: 21
hot pixels, rgb: 24
dark pixels: 46


In [47]:
out_path = "E:/Focus stacking/2025-04-19 - Hemiptera/A/noise-map/"
cv2.imwrite(out_path + "hot_b.png", hot_b)
cv2.imwrite(out_path + "hot_g.png", hot_g)
cv2.imwrite(out_path + "hot_r.png", hot_r)
cv2.imwrite(out_path + "hot_rgb.png", hot_rgb)
cv2.imwrite(out_path + "dark.png", mask_dark)
cv2.imwrite(out_path + "mean_img.tif", mean_img)

True

In [40]:
image = cv2.imread(input_path + "/_MG_8521.tif")
defect_mask = cv2.imread(out_path + "hot_rgb.png", cv2.IMREAD_GRAYSCALE)  # Maschera binaria
corrected_image = correct_defective_pixels(image, defect_mask, kernel_size=5, method='median')
cv2.imwrite(out_path + "orig.tif", image)
cv2.imwrite(out_path + "corr.tif", corrected_image)

True